# THỐNG KÊ DỮ LIỆU

1. Yêu cầu chung: Thống kê được số lượng ảnh trong từng phân loại mà mỗi SV đóng góp

2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/1Uj0V9URNHpzSHeXHSB89AoGCjGki8Yra
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước https://colab.research.google.com/drive/1bUmXMM_ggnEXKo2qylfe6h0JnQuos-8_
- Output:
    + File CarDataset-1.csv - Thống kê số lượng ảnh theo từng SV
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: MSSV, All, Số lượng.
        
        Ví dụ: 20221234, All, 1.2
      + Số lượng có thể là số thực vì một ảnh có thể có đóng góp từ nhiều sinh viên
    + File CarDataset-2.csv - Thống kê số lượng ảnh theo từng Hiệu xe mà từng SV đóng góp
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: MSSV, Hiệu xe, Số lượng.
        
        Ví dụ: 20221234, Honda, 1.2
      + Số lượng có thể là số thực vì một ảnh có thể có đóng góp từ nhiều sinh viên
      + Chỉ chấp nhận file .jpg, .jpeg và .png
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: 17:00 - ??/??/2024

4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.


In [1]:
import os
import csv
from collections import defaultdict

In [2]:
def extract_info(file_name):
    """
    Trích xuất thông tin từ tên tệp.
    
    Args:
        file_name (str): Tên tệp (file) ảnh.

    Returns:
        tuple: Bao gồm danh sách MSSV, thương hiệu xe (brand), và chỉ số đóng góp (contribution index).
    """
    try:
        parts = file_name.split('.')
        if len(parts) < 3:
            return None, None, None

        mssv_part, brand, _ = parts[0], parts[1], parts[2]
        mssvs = mssv_part.split('-') 
        return mssvs, brand, 1 / len(mssvs)  
    except Exception:
        return None, None, None

In [3]:
def process_images(parent_folder):
    """
    Xử lý thư mục chứa ảnh và tính toán đóng góp của từng sinh viên.

    Args:
        parent_folder (str): Đường dẫn tới thư mục cha chứa các thư mục ảnh theo thương hiệu.

    Returns:
        tuple:
            - contributions_by_student (dict): Tổng số lượng ảnh đóng góp bởi từng MSSV.
            - contributions_by_student_and_brand (dict): Số lượng ảnh đóng góp theo MSSV và thương hiệu xe.
    """
    contributions_by_student = defaultdict(float)
    
    contributions_by_student_and_brand = defaultdict(lambda: defaultdict(float))

    valid_extensions = {".jpg", ".jpeg", ".png"}

    for brand_folder in os.listdir(parent_folder):
        brand_path = os.path.join(parent_folder, brand_folder)  
        if not os.path.isdir(brand_path):
            continue 

        for file_name in os.listdir(brand_path):
            if not any(file_name.lower().endswith(ext) for ext in valid_extensions):
                continue  
            mssvs, brand, contribution = extract_info(file_name)
            if not mssvs or not brand:
                continue

            for mssv in mssvs:
                contributions_by_student[mssv] += contribution 
                contributions_by_student_and_brand[mssv][brand] += contribution 

    return contributions_by_student, contributions_by_student_and_brand

In [4]:
def save_to_csv(file_path, data, headers):
    """
    Lưu dữ liệu vào tệp CSV.

    Args:
        file_path (str): Đường dẫn lưu tệp CSV.
        data (list): Dữ liệu cần lưu, dạng danh sách các hàng.
        headers (list): Danh sách các tiêu đề cột.
    """
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  
        writer.writerows(data)  

In [5]:
def main():
    """
    Hàm chính để xử lý toàn bộ quy trình:
    - Tính toán đóng góp của sinh viên từ ảnh.
    - Lưu dữ liệu vào các tệp CSV.
    """
    
    parent_folder = r"D:\ML\Public"  

    contributions_by_student, contributions_by_student_and_brand = process_images(parent_folder)

    dataset1 = [(mssv, "All", round(total, 2)) for mssv, total in contributions_by_student.items()]
    save_to_csv("CarDataset-1.csv", dataset1, ["MSSV", "All", "Quantity"])

    dataset2 = [
        (mssv, brand, round(quantity, 2))
        for mssv, brands in contributions_by_student_and_brand.items()
        for brand, quantity in brands.items()
    ]
    save_to_csv("CarDataset-2.csv", dataset2, ["MSSV", "Brand", "Quantity"])

if __name__ == "__main__":
    main()